# Conjunto BB1
Este conjunto de datos posee la misma señal que el conjunto R&D, pero con un 0,08 % de señal. También hay diferencia en las masas de las partículas $Z'$, $X$ y $Y$, que para este conjunto son 3,823 TeV, 732 GeV y 378 GeV, respectivamente. 

In [1]:
# Importamos las librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from myst_nb import glue

# Funciones de benchtools
from benchtools.src.plotools import bkg_sig_hist, bkg_sig_scatter
from benchtools.src.clustering import build_features

In [3]:
# Pre-procesando datos
path_data = "../../../datos/events_LHCO2020_BlackBox1.h5"
path_key = "../../../datos/events_LHCO2020_BlackBox1.masterkey"
build_features(path_data=path_data, nbatch=2, outname='BB1-100000', path_label=path_key, outdir='../../../datos/', chunksize=50000)

Part 1/2


100%|██████████| 50000/50000 [12:50<00:00, 64.93it/s]


Part 2/2


100%|██████████| 50000/50000 [11:48<00:00, 70.54it/s]


Done
Merging files


In [6]:
df = pd.read_csv("../../../datos/BB1-100000.csv")

# Calculamos la proporción de señal
df_sig = df[df.iloc[:,-1]==1]    # Última columna es señal
ratio_sig = df_sig.shape[0]*100/df.shape[0]
# Guardamos el valor en el notebook
glue("BB1-ratio", ratio_sig, display=False)

Para analizar este conjunto de datos, también utilizamos una fracción de los datos. 100,000 eventos, con {glue:text}`BB1-ratio:.2f`% de señal, que es una buena representación del conjunto completo.
## Datos pre-procesados
Los datos pre-procesados de este conjunto presentan las mismas características que los del conjunto R&D. La diferencia principal son las distribuciones de masa.